In [1]:
import pandas as pd
import os
desktop_path = os.path.expanduser("~/Desktop")
file_path = os.path.join(desktop_path, "random_subset.xlsx")
df = pd.read_excel(file_path)
print(df.head())

   UMID                                     cleaned_review sentiment  \
0  AE_1  ms files left breathless got educated got mad ...  Positive   
1  AE_2                                          fast card  Negative   
2  AE_3  heavy dutyeasy installand important easy attac...  Positive   
3  AE_4  summer breeze second book series collection ba...  Positive   
4  AE_5               outstanding value great product look  Positive   

  predicted_category  
0               Care  
1            General  
2              Leads  
3            General  
4              Leads  


In [2]:
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.models import Model

2023-08-22 21:41:15.863913: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-22 21:41:18.763593: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-22 21:41:18.775449: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-22 21:41:27.675489: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
reviews = df['cleaned_review'].tolist()
sentiments = df['sentiment'].tolist()
categories = df['predicted_category'].tolist()

In [4]:
sentiment_mapping = {'Positive': 0, 'Negative': 1, 'Neutral': 2}
category_mapping = {'Care': 0, 'General': 1, 'Leads': 2}

In [5]:
sentiment_labels = [sentiment_mapping[label] for label in sentiments]
category_labels = [category_mapping[label] for label in categories]

In [6]:
max_words = 10000  # Choose an appropriate vocabulary size
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(reviews)
sequences = tokenizer.texts_to_sequences(reviews)
max_sequence_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post')

In [7]:
train_reviews, val_reviews, train_sentiments, val_sentiments, train_categories, val_categories = train_test_split(
    padded_sequences, sentiment_labels, category_labels, test_size=0.2, random_state=42
)

In [8]:
# Define the model
input_layer = Input(shape=(max_sequence_length,))
embedding_layer = Embedding(input_dim=max_words, output_dim=128)(input_layer)
lstm_layer = LSTM(64)(embedding_layer)

2023-08-22 21:41:36.965945: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-08-22 21:41:36.968905: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-08-22 21:41:36.971211: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [9]:
# Separate output layers for sentiment and category
num_sentiments = len(sentiment_mapping)
num_categories = len(category_mapping)

In [10]:
sentiment_output = Dense(num_sentiments, activation='softmax', name='sentiment')(lstm_layer)
category_output = Dense(num_categories, activation='softmax', name='category')(lstm_layer)

In [11]:
model = Model(inputs=input_layer, outputs=[sentiment_output, category_output])

In [12]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [13]:
train_sentiments = np.array(train_sentiments)
train_categories = np.array(train_categories)
val_sentiments = np.array(val_sentiments)
val_categories = np.array(val_categories)

In [14]:
# Train the model
model.fit(train_reviews, [train_sentiments, train_categories], epochs=10, batch_size=32,
          validation_data=(val_reviews, [val_sentiments, val_categories]))

Epoch 1/10


2023-08-22 21:41:38.775252: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-08-22 21:41:38.777646: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-08-22 21:41:38.779605: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

3/3 [==============================] - ETA: 0s - loss: 2.1784 - sentiment_loss: 1.0784 - category_loss: 1.1000 - sentiment_accuracy: 0.4625 - category_accuracy: 0.3500

2023-08-22 21:41:43.289421: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-08-22 21:41:43.291986: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-08-22 21:41:43.293833: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

3/3 [==============================] - 5s 497ms/step - loss: 2.1784 - sentiment_loss: 1.0784 - category_loss: 1.1000 - sentiment_accuracy: 0.4625 - category_accuracy: 0.3500 - val_loss: 2.0755 - val_sentiment_loss: 1.0159 - val_category_loss: 1.0596 - val_sentiment_accuracy: 0.8000 - val_category_accuracy: 0.6500
Epoch 2/10
3/3 [==============================] - 0s 173ms/step - loss: 2.0204 - sentiment_loss: 0.9914 - category_loss: 1.0289 - sentiment_accuracy: 0.8250 - category_accuracy: 0.7625 - val_loss: 1.9257 - val_sentiment_loss: 0.9258 - val_category_loss: 0.9999 - val_sentiment_accuracy: 0.8000 - val_category_accuracy: 0.6500
Epoch 3/10
3/3 [==============================] - 0s 172ms/step - loss: 1.8219 - sentiment_loss: 0.8848 - category_loss: 0.9372 - sentiment_accuracy: 0.8250 - category_accuracy: 0.7625 - val_loss: 1.7284 - val_sentiment_loss: 0.8039 - val_category_loss: 0.9245 - val_sentiment_accuracy: 0.8000 - val_category_accuracy: 0.6500
Epoch 4/10
3/3 [=================

In [15]:
# Save the model
model.save('multi_output_model.h5')
print("Model saved successfully")

Model saved successfully


## Validation

In [17]:
excel_path = '/home/xjoex/Desktop/review_text.xlsx'
df_loaded = pd.read_excel(excel_path)
print(df_loaded.head())

                                          reviewText  \
0  This was the first time I read Garcia-Aguilera...   
1  As with all of Ms. Garcia-Aguilera's books, I ...   
2  I've not read any of Ms Aguilera's works befor...   
3  This romance novel is right up there with the ...   
4  Carolina Garcia Aguilera has done it again.  S...   

                                      cleaned_review  
0  first time read garciaaguilera came upon name ...  
1  ms garciaaguileras books think must read impos...  
2  ive read ms aguileras works finished one hot s...  
3  romance novel right rest amazing mystery novel...  
4  carolina garcia aguilera done shes written ano...  


In [18]:
# Preprocess text data from df_loaded
reviews_subset = df_loaded['cleaned_review'].tolist()[:200] 
sequences_subset = tokenizer.texts_to_sequences(reviews_subset)
padded_sequences_subset = pad_sequences(sequences_subset, maxlen=max_sequence_length, padding='post')

In [19]:
sentiment_probs, category_probs = model.predict(padded_sequences_subset)

1/7 [===>..........................] - ETA: 2s

2023-08-22 21:47:02.651471: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-08-22 21:47:02.653847: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-08-22 21:47:02.655703: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

7/7 [==============================] - 1s 40ms/step


In [20]:
# Convert probabilities to labels
predicted_sentiments = np.argmax(sentiment_probs, axis=1)
predicted_categories = np.argmax(category_probs, axis=1)

In [21]:
reverse_sentiment_mapping = {v: k for k, v in sentiment_mapping.items()}
reverse_category_mapping = {v: k for k, v in category_mapping.items()}

In [22]:
predicted_sentiment_labels = [reverse_sentiment_mapping[sentiment] for sentiment in predicted_sentiments]
predicted_category_labels = [reverse_category_mapping[category] for category in predicted_categories]

In [23]:
predictions_df = pd.DataFrame({
    'Review': reviews_subset,
    'Predicted_Sentiment': predicted_sentiment_labels,
    'Predicted_Category': predicted_category_labels
})

In [24]:
predictions_df.head(10)

,Review,Predicted_Sentiment,Predicted_Category
0,first time read garciaaguilera came upon name ...,Positive,General
1,ms garciaaguileras books think must read impos...,Positive,General
2,ive read ms aguileras works finished one hot s...,Positive,General
3,romance novel right rest amazing mystery novel...,Positive,General
4,carolina garcia aguilera done shes written ano...,Positive,General
5,write mysteriesbut sure write love story one h...,Positive,General
6,garciaaguilera written book cant put read love...,Positive,General
7,timing right good book actually long overdue t...,Positive,General
8,engaging dark reading book could almost feel t...,Positive,General
9,set amid backdrop new york citys impervious hi...,Positive,General


In [25]:
desktop_path = f"/home/xjoex/Desktop"  
predictions_excel_filename = "predictions.xlsx"
predictions_excel_path = f"{desktop_path}/{predictions_excel_filename}"

predictions_df.to_excel(predictions_excel_path, index=False)

print(f"Predictions saved to: {predictions_excel_path}")

Predictions saved to: /home/xjoex/Desktop/predictions.xlsx


## Developing a Semi-Supervised Neural Network

In [26]:
import pandas as pd
file_path = '/home/xjoex/Desktop/review_text.xlsx'
df_review_text = pd.read_excel(file_path)
print(df_review_text.head())

                                          reviewText  \
0  This was the first time I read Garcia-Aguilera...   
1  As with all of Ms. Garcia-Aguilera's books, I ...   
2  I've not read any of Ms Aguilera's works befor...   
3  This romance novel is right up there with the ...   
4  Carolina Garcia Aguilera has done it again.  S...   

                                      cleaned_review  
0  first time read garciaaguilera came upon name ...  
1  ms garciaaguileras books think must read impos...  
2  ive read ms aguileras works finished one hot s...  
3  romance novel right rest amazing mystery novel...  
4  carolina garcia aguilera done shes written ano...  


In [28]:
unlabeled_reviews = df_review_text['cleaned_review'].astype(str).tolist()
unlabeled_sequences = tokenizer.texts_to_sequences(unlabeled_reviews)
padded_unlabeled_sequences = pad_sequences(unlabeled_sequences, maxlen=max_sequence_length, padding='post')

In [29]:
# Pseudo-label the unlabeled data
unlabeled_sentiments_probs, unlabeled_categories_probs = model.predict(padded_unlabeled_sequences)
pseudo_sentiments = np.argmax(unlabeled_sentiments_probs, axis=1)
pseudo_categories = np.argmax(unlabeled_categories_probs, axis=1)

    1/32713 [..............................] - ETA: 54:43

2023-08-22 21:55:02.465508: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1076114512 exceeds 10% of free system memory.


32713/32713 [==============================] - 1288s 39ms/step
